In [1]:
import pandas as pd
import ast

In [3]:
df_stops_total = pd.read_csv('../../final-tables/stations.csv')
df_1946 = pd.read_csv('../initial/stops_df_1946-initial.csv')

In [4]:
df_1946.head(3)

,Unnamed: 0,stop_name,line_count,type,in_lines
0,0,Flughafen,1,bus,['1']
1,1,Franz-Mehring-Strasse,3,bus,['1']
2,2,Franz-Mehring-Strasse,3,strassenbahn,['99']


In [5]:
df_1946.drop(columns=['Unnamed: 0'], inplace=True)
df_1946["location"] = ""
df_1946["identifier"] = ""
df_1946["stop_id_other"] = ""
df_1946["previous_in_lines"] = ""
df_1946["stop_description"] = ""

In [7]:
df_stops_total.tail(3)

,stop_id,stop_name,type,location,in_lines,identifier,stop_description
6423,19641453,Zum Heckeshorn,bus,"52.43307505069834, 13.164008736441371",A51,NaN,Zum Heckeshorn Ecke Am Großen Wannsee
6424,19641454,Zweibrücker Strasse,bus,"52.54677409100436, 13.188764018875105",A63,NaN,NaN
6425,19641455,Zühlsdorf,bus,"52.7162250652868, 13.366925671584623",AS,NaN,NaN


In [6]:
# Function to convert string representation of set to list
def convert_to_list(value):
    try:
        # Use ast.literal_eval to safely convert string to set
        items = ast.literal_eval(value)
        
        # Check if the result is a set, and convert it to a list
        if isinstance(items, set):
            return list(items)
        # Check if the result is an integer, and convert it to a list with a single element
        elif isinstance(items, int):
            return [items]
        # For other types, return a list with the original value
        else:
            return [value]
    except (SyntaxError, ValueError):
        # Handle the case where literal_eval fails (e.g., invalid string)
        # Try to convert the value to an integer, and return a list with a single element if successful
        try:
            return [int(value)]
        except ValueError:
            return [value]

# Apply the function to the 'in_lines' column
df_stops_total['in_lines'] = df_stops_total['in_lines'].apply(convert_to_list)
df_1946['in_lines'] = df_1946['in_lines'].apply(convert_to_list)

In [7]:
df_stops_total['in_lines'] = df_stops_total['in_lines'].apply(lambda x: ','.join(map(str, x)))
df_1946['in_lines'] = df_1946['in_lines'].apply(lambda x: ','.join(map(str, x)))

In [8]:
df_stops_total.head(3)

,stop_id,stop_name,type,location,in_lines,identifier,stop_description
0,19460,Adlershof,s-bahn,"52.434722222222,13.541388888889","['KBS 100a', 'KBS 100c']",Q323551,NaN
1,19461,Akazienallee,bus,"52.47849809201335, 13.607179083668733",['D'],NaN,NaN
2,19462,Alexanderplatz,bus,"52.52005246558073, 13.413957243449447","['1', '9']",NaN,NaN


I have two dataframes df_long and df_short, I want to add the information from df_long columns combined_location and combined_identifier to df_short if there is a match between the original_name column from df_long with the stop_name column of df_short. I have to create combined_location and combined_identifier columns in df_short first

In [ ]:
for index, row in df_1946.iterrows():
    # Check if row["stop_name"] is in df_1960["original_name"]
    if row["stop_name"] in df_stops_total["stop_name"].values:
        # Find matching rows in df_1960
        matching_rows = df_stops_total[df_stops_total["stop_name"] == row["stop_name"]]
        
        # Check if those two rows have matching row["type"] values
        matching_rows = matching_rows[matching_rows["type"] == row["type"]]
        
        # Check if there are any matching rows
        if not matching_rows.empty:
            df_1946.at[index, 'location'] = '+ '.join(matching_rows['location'].astype(str).values)
            df_1946.at[index, 'identifier'] = '+ '.join(matching_rows['identifier'].astype(str).values)
            df_1946.at[index, 'stop_description'] = '+ '.join(matching_rows['stop_description'].astype(str).values)
            df_1946.at[index, 'stop_id_other'] = '+ '.join(matching_rows['stop_id'].astype(str).values)
            df_1946.at[index, 'previous_in_lines'] = '+ '.join(matching_rows['in_lines'].astype(str).values)
        else:

            df_1946.at[index, 'location'] = ''
            df_1946.at[index, 'identifier'] = ''
            df_1946.at[index, 'stop_description'] = ''
            df_1946.at[index, 'stop_id_other'] = ''
            df_1946.at[index, 'previous_in_lines'] = ''

In [12]:
# Function to split string values containing '+' and convert to a unique string
def split_values(cell_value):
    unique_values = sorted(set(element.strip() for element in cell_value.split('+')))
    return '+'.join(unique_values)

# Apply the function to each cell in the specified columns
df_1946['location'] = df_1946['location'].apply(split_values)
df_1946['identifier'] = df_1946['identifier'].apply(split_values)

In [13]:
df_1946.drop(columns=['line_count', "stop_id_other"], inplace=True)

In [14]:
# Function to concatenate unique values in other columns
def concat_unique_values(series):
    unique_values = set(series)
    return '+ '.join(unique_values)

# Group by specified columns and aggregate the other columns
df_1946 = df_1946.groupby(['stop_name', 'type', 'in_lines', 'location']).agg({
    'identifier': concat_unique_values,
    'previous_in_lines': concat_unique_values,
    'stop_description': concat_unique_values
}).reset_index()

In [15]:
df_1946.to_csv('1946-stops-combined.csv')

In [16]:
df_1946.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688 entries, 0 to 687
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   stop_name          688 non-null    object
 1   type               688 non-null    object
 2   in_lines           688 non-null    object
 3   location           688 non-null    object
 4   identifier         688 non-null    object
 5   previous_in_lines  688 non-null    object
 6   stop_description   688 non-null    object
dtypes: object(7)
memory usage: 37.8+ KB


In [17]:
df_1946.head(10)

,stop_name,type,in_lines,location,identifier,previous_in_lines,stop_description
0,Adlershof,s-bahn,"['KBS 100a', 'KBS 100c']","52.434722222222,13.541388888889",Q323551,"['KBS 100a', 'KBS 103']+ ['KBS 100a', 'KBS 103...",nan+ nan+ Berlin-Adlershof station+ Berlin-Adl...
1,Akazienallee,bus,['D'],,,,
2,Alexanderplatz,bus,"['1', '9']",,,,
3,Alexanderplatz,s-bahn,"['KBS 101', 'KBS 100c']","52.521388888889,13.411944444444",Q111324700,"['KBS 101', 'KBS 103', 'KBS 102']+ ['KBS 102',...",nan+ nan+ S-Bahnhof Berlin Alexanderplatz+ S-B...
4,Alexanderplatz,strassenbahn,"['72', '64', '73', '65', '60']","52.5219449,13.4143847",Q65548996,['1'],nan
5,Alexanderplatz,u-bahn,"['E', 'A', 'D']","52.5214,13.4119",Q17173458,"['A I', 'D', 'E']+ ['D', 'E', 'A I']+ E,D,A I,...",nan+ nan+ Alexanderplatz u-bahn station+ Alexa...
6,Alexanderstrasse,bus,['1'],,,,
7,Alexanderstrasse,strassenbahn,['82'],,,,
8,Alt-Biesdorf,bus,['H'],,,,
9,Alt-Friedrichsfelde,bus,"['H', '1']",,,,
